# Define model

In [19]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten, Permute, Reshape
from keras.layers import Activation
from keras import backend as K
 
class LeNet:
    @staticmethod
    def build(width, height, depth, classes):
        # initialize the model
        model = Sequential()
        inputShape = (height, width, depth)
 
        # if we are using "channels first", update the input shape
        #if K.image_data_format() == "channels_first":
        #inputShape = (depth, height, width)
        # first set of CONV => RELU => POOL layers
        model.add(Conv2D(20, (5, 5), padding="same",input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        # second set of CONV => RELU => POOL layers
        model.add(Conv2D(50, (5, 5), padding="same"))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

        # first (and only) set of FC => RELU layers
        a, b, c, d = model.output_shape
        a = b * c * d
        model.add(Permute([1, 2, 3]))  # Indicate NHWC data layout
        model.add(Reshape((a,)))
        #model.add(Flatten())
        model.add(Dense(500))
        model.add(Activation("relu"))

        # softmax classifier
        model.add(Dense(classes))
        model.add(Activation("softmax"))

        # return the constructed network architecture
        return model

In [20]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
 
# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
#from pyimagesearch.lenet import LeNet
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import cv2
import os

In [21]:
# initialize the number of epochs to train for, initial learning rate,
# and batch size
EPOCHS = 25
INIT_LR = 1e-3
BS = 2
 
# initialize the data and labels
print("[INFO] loading images...")
data = []
labels = []
 
# grab the image paths and randomly shuffle them
imagePaths = sorted(list(paths.list_images('training_data')))
random.seed(42)
random.shuffle(imagePaths)

[INFO] loading images...


In [22]:
imagePaths

['training_data\\Not_Boost\\Not_Boost_3.png',
 'training_data\\Boost\\Boost_4.png',
 'training_data\\Boost\\Boost_3.png',
 'training_data\\Not_Boost\\Not_Boost_4.png',
 'training_data\\Not_Boost\\Not_Boost_1.png',
 'training_data\\Not_Boost\\Not_Boost_2.png',
 'training_data\\Not_Boost\\Not_Boost_5.png',
 'training_data\\Boost\\Boost_5.png',
 'training_data\\Boost\\Boost_1.png',
 'training_data\\Boost\\Boost_2.png']

In [23]:
for imagePath in imagePaths:
    # load the image, pre-process it, and store it in the data list
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (28, 28))
    image = img_to_array(image)
    data.append(image)

    # extract the class label from the image path and update the
    # labels list
    label = imagePath.split(os.path.sep)[-2]
    label = 1 if label == "Boost" else 0
    labels.append(label)

In [24]:
labels

[0, 1, 1, 0, 0, 0, 0, 1, 1, 1]

In [25]:
# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)
 
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data,labels, test_size=0.25, random_state=42)
 
# convert the labels from integers to vectors
trainY = to_categorical(trainY, num_classes=2)
testY = to_categorical(testY, num_classes=2)

In [26]:
# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,horizontal_flip=True, fill_mode="nearest")

In [27]:
# initialize the model
print("[INFO] compiling model...")
model = LeNet.build(width=28, height=28, depth=3, classes=2)
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])
 
# train the network
print("[INFO] training network...")
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BS),validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS, epochs=EPOCHS, verbose=1)
#H = model.fit_generator(trainX, trainY,validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS, epochs=EPOCHS, verbose=1)
 
# save the model to disk
#print("[INFO] serializing network...")
#model.save(args["model"])

[INFO] compiling model...
[INFO] training network...
Epoch 1/25
3/3 [==============================] - 0s 58ms/step - loss: 0.7228 - accuracy: 0.5000 - val_loss: 0.9559 - val_accuracy: 0.3333
Epoch 2/25
3/3 [==============================] - 0s 67ms/step - loss: 0.8334 - accuracy: 0.4000 - val_loss: 0.5626 - val_accuracy: 1.0000
Epoch 3/25
3/3 [==============================] - 0s 30ms/step - loss: 0.5736 - accuracy: 0.6000 - val_loss: 0.4952 - val_accuracy: 0.6667
Epoch 4/25
3/3 [==============================] - 0s 6ms/step - loss: 0.6123 - accuracy: 0.4000 - val_loss: 0.4388 - val_accuracy: 0.6667
Epoch 5/25
3/3 [==============================] - 0s 62ms/step - loss: 0.5115 - accuracy: 0.6667 - val_loss: 0.3567 - val_accuracy: 1.0000
Epoch 6/25
3/3 [==============================] - 0s 39ms/step - loss: 0.4835 - accuracy: 1.0000 - val_loss: 0.2720 - val_accuracy: 1.0000
Epoch 7/25
3/3 [==============================] - 0s 65ms/step - loss: 0.3542 - accuracy: 1.0000 - val_loss: 0.201

# Predict

In [33]:
# grab the image paths and randomly shuffle them
validate_path = sorted(list(paths.list_images('validate')))
#random.seed(42)
#random.shuffle(imagePaths)

In [34]:
validate_path

['validate\\Boost_1.png']

In [35]:
val_data = [] 

In [36]:
for imagePath in validate_path:
    # load the image, pre-process it, and store it in the data list
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (28, 28))
    image = img_to_array(image)
    val_data.append(image)

In [37]:
val_data = np.array(val_data, dtype="float") / 255.0

In [38]:
pred_prob = model.predict(val_data)

In [39]:
pred_prob

array([[5.059523e-09, 1.000000e+00]], dtype=float32)

In [40]:
np.argmax(pred_prob)

1

# Export keras model by pb

In [41]:
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.
 
    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session 
    to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                      output_names, freeze_var_names)
        return frozen_graph

In [42]:
from keras import backend as K
import tensorflow as tf
 
# Create, compile and train model...
 
frozen_graph = freeze_session(K.get_session(),
                              output_names=[out.op.name for out in model.outputs])

W0103 19:41:15.521055  7408 deprecation.py:323] From <ipython-input-41-ea683daeba82>:28: convert_variables_to_constants (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
W0103 19:41:15.521055  7408 deprecation.py:323] From C:\Users\thomas\Anaconda3\lib\site-packages\tensorflow_core\python\framework\graph_util_impl.py:277: extract_sub_graph (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


In [43]:
tf.train.write_graph(frozen_graph, "model", "my_model.pb", as_text=False)

'model\\my_model.pb'